In [1]:
import sys
print(sys.executable)

/Users/andykim/Documents/2025 Fall/NNDL/Project/NNDL_MRI_PROJECT/venv/bin/python


In [2]:
import splitfolders
import os

# Assign 4 classes of MRI images (MildDemented, ModerateDemented, NonDemented, VeryMildDemented) randomly and evenly as train, validation, and test data
splitfolders.ratio(
    "/Users/andykim/Documents/2025 Fall/NNDL/Project/NNDL_MRI_PROJECT/Augmented_Data",
    output="/Users/andykim/Documents/2025 Fall/NNDL/Project/NNDL_MRI_PROJECT/Split_Data",
    seed=1337,
    ratio=(.8, .1, .1)
)

print("Split complete")

# Check the distribution of MRI images

split_data_path = "/Users/andykim/Documents/2025 Fall/NNDL/Project/NNDL_MRI_PROJECT/Split_Data"

print("=" * 80)
print("IMAGE COUNT PER SPLIT PER CLASS")
print("=" * 80)

for split in ['train', 'val', 'test']:
    split_path = os.path.join(split_data_path, split)
    
    if not os.path.exists(split_path):
        print(f"\n{split.upper()}: Folder not found!")
        continue
    
    print(f"\n{split.upper()}:")
    print("-" * 80)
    
    total = 0
    classes = sorted(os.listdir(split_path))
    
    for class_name in classes:
        if class_name == '.DS_Store':
            continue
            
        class_path = os.path.join(split_path, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if not f.startswith('.')])
            total += count
            print(f"  {class_name:50s} {count:6d} images")
    
    print(f"  {'-' * 50}")
    print(f"  {'TOTAL':50s} {total:6d} images")

print("\n" + "=" * 80)

# Summary table
print("\nSUMMARY TABLE:")
print("=" * 80)
print(f"{'Class':<50s} {'Train':>8s} {'Val':>8s} {'Test':>8s} {'Total':>8s}")
print("-" * 80)

classes = []
for split in ['train', 'val', 'test']:
    split_path = os.path.join(split_data_path, split)
    if os.path.exists(split_path):
        classes = sorted([c for c in os.listdir(split_path) if c != '.DS_Store' and os.path.isdir(os.path.join(split_path, c))])
        break

full_total = 0
for class_name in classes:
    counts = []
    for split in ['train', 'val', 'test']:
        class_path = os.path.join(split_data_path, split, class_name)
        if os.path.exists(class_path):
            count = len([f for f in os.listdir(class_path) if not f.startswith('.')])
            counts.append(count)
        else:
            counts.append(0)
    
    total = sum(counts)
    print(f"{class_name:<50s} {counts[0]:8d} {counts[1]:8d} {counts[2]:8d} {total:8d}")
    full_total += total
print("-" * 80)
print("Total count across all classes: ", full_total)
print("=" * 80)

Copying files: 6735 files [00:01, 3428.20 files/s]

Split complete
IMAGE COUNT PER SPLIT PER CLASS

TRAIN:
--------------------------------------------------------------------------------
  Brain - Alzheimer MildDemented                       1399 images
  Brain - Alzheimer ModerateDemented                   1139 images
  Brain - Alzheimer NonDemented                        1653 images
  Brain - Alzheimer VeryMildDemented                   1196 images
  --------------------------------------------------
  TOTAL                                                5387 images

VAL:
--------------------------------------------------------------------------------
  Brain - Alzheimer MildDemented                        174 images
  Brain - Alzheimer ModerateDemented                    142 images
  Brain - Alzheimer NonDemented                         206 images
  Brain - Alzheimer VeryMildDemented                    149 images
  --------------------------------------------------
  TOTAL                                                 671 images



In [ ]:
# Run the following command 
pip install numpy pandas scikit-learn matplotlib seaborn keras tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.applications.resnet import ResNet50 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
channels=3
batch_size=25
test_batch_size=32 
test_steps=1
train_path = './Split_Data/train'
test_path = './Split_Data/test'
val_path = './Split_Data/val'
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps)
def scalar(img):    
    return img  # EfficientNet expects pixelsin range 0 to 255 so no scaling is required
trgen=ImageDataGenerator(preprocessing_function=scalar, horizontal_flip=True)
tvgen=ImageDataGenerator(preprocessing_function=scalar)
train_generator=trgen.flow_from_directory( directory=train_path , target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)
test_generator=tvgen.flow_from_directory( directory=test_path, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=False, batch_size=test_batch_size)

valid_generator=tvgen.flow_from_directory( directory=val_path, target_size=(224,224), class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)
classes=list(train_generator.class_indices.keys())
class_count=len(classes)
train_steps=int(np.ceil(len(train_generator.labels)/batch_size))

test batch size:  32   test steps:  1
Found 5387 images belonging to 4 classes.
Found 677 images belonging to 4 classes.
Found 671 images belonging to 4 classes.
